## 마켓플레이스 모델 구독

라이브러리

In [1]:
# AWS 관련
import sagemaker
import boto3

# 시각화 및 이미지 관련
import plotly.graph_objects as go
from io import BytesIO
from PIL import Image

# 기타 유틸리티
import requests
import os
import json
from dotenv import load_dotenv
load_dotenv()

sagemaker.config INFO - Not applying SDK defaults from location: /Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/dante/Library/Application Support/sagemaker/config.yaml


True

SDK 클라이언트 설정

In [6]:
boto3_session = boto3.Session(profile_name='awstutor')
sagemaker_session = sagemaker.Session(boto_session=boto3_session)
role = os.environ.get('SAGEMAKER_EXECUTION_ROLE_ARN')
sm_runtime = boto3_session.client(service_name='sagemaker-runtime')

헬퍼 함수 정의

In [7]:
# 이미지 URL을 인코딩하는 함수
def encode_image_url(image_url):
    response = requests.get(image_url)
    image_content = response.content
    return image_content

In [8]:
# 결과 시각화
def visualize_detection_results(encoded_image, result):
    # 이미지 로드
    image = Image.open(BytesIO(encoded_image))

    # 이미지를 plotly 형식으로 변환
    fig = go.Figure()

    fig.add_trace(go.Image(z=image))

    # 검출된 객체에 대한 바운딩 박스 그리기
    for obj in result:
        fig.add_shape(
            type="rect",
            x0=obj['left'], y0=obj['top'],
            x1=obj['right'], y1=obj['bottom'],
            line=dict(color="red", width=2),
            fillcolor="rgba(255,0,0,0.1)"
        )
        fig.add_annotation(
            x=obj['left'], y=obj['top'],
            text=f"{obj['id']}: {obj['score']:.2f}",
            showarrow=False,
            font=dict(color="white", size=10),
            bgcolor="red",
            opacity=0.7
        )

    # 레이아웃 설정
    fig.update_layout(
        showlegend=False,
        width=800,
        height=600,
        margin=dict(l=0, r=0, t=0, b=0)
    )

    # 축 숨기기
    fig.update_xaxes(visible=False, range=[0, image.width])
    fig.update_yaxes(visible=False, range=[image.height, 0])
    fig.show()
    return fig

In [ ]:
# 객체 검출 함수
def detect_object(img_url):
    # 이미지 인코딩
    encoded_image = encode_image_url(img_url)
    # 엔드포인트 호출
    response = sm_runtime.invoke_endpoint(
        EndpointName='dante-yolov3-obj-detector-endpoint',
        ContentType="image/jpeg",
        Body=encoded_image,
        CustomAttributes='{"threshold": 0.2}',
        Accept='application/json'
    )
    # 결과 파싱
    result = json.loads(response['Body'].read().decode())
    # 결과 시각화
    visualize_detection_results(encoded_image, result)

실시간 추론

In [ ]:
detect_object('https://cdn.imweb.me/thumbnail/20240904/6640655fd55af.jpeg')

In [ ]:
detect_object('https://cdn.imweb.me/thumbnail/20240904/a4337fba7cb6f.jpeg')

In [ ]:
detect_object('https://cdn.imweb.me/thumbnail/20240904/37c449ac520f3.jpeg')

리소스 정리

In [39]:
# 엔드포인트 삭제
sm_client = boto3_session.client(service_name='sagemaker')
sm_client.delete_endpoint(EndpointName='dante-yolov3-obj-detector-endpoint')
print('엔드포인트가 성공적으로 삭제되었습니다.')

엔드포인트가 성공적으로 삭제되었습니다.


In [41]:
# 엔드포인트 구성 삭제
sm_client.delete_endpoint_config(EndpointConfigName='dante-yolov3-obj-detector-endpoint-config')
print('엔드포인트 구성이 성공적으로 삭제되었습니다.')

엔드포인트 구성이 성공적으로 삭제되었습니다.


In [43]:
# 모델 삭제
sm_client.delete_model(ModelName='dante-yolo3-obj-detector')
print('모델이 성공적으로 삭제되었습니다.')

모델이 성공적으로 삭제되었습니다.


: 